In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Enter verification code: 4/1AWtgzh6v4w2apatsjfih7vo4ixLoCLQa-beRL6bCBhqgbmpYSovTz1O1pIg

Successfully saved authorization token.


In [14]:
Hansen_GFC = ee.Image("UMD/hansen/global_forest_change_2020_v1_8")
SRTM = ee.Image("USGS/SRTMGL1_003")
surfaceWater = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
#imperviousSurface = ee.Image("Tsinghua/FROM-GLC/GAIA/v10")
#GAUL_adm1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
GFSAD30AFCE_2015_KEN_1 = ee.Image("projects/ee-aboubacarhema94/assets/Kenya/GFSAD30AFCE_2015_KEN_1")
GFSAD30AFCE_2015_KEN_2 = ee.Image("projects/ee-aboubacarhema94/assets/Kenya/GFSAD30AFCE_2015_KEN_2")
GFSAD30AFCE_2015_KEN_3 = ee.Image("projects/ee-aboubacarhema94/assets/Kenya/GFSAD30AFCE_2015_KEN_3")
GFSAD30AFCE_2015_KEN_4 = ee.Image("projects/ee-aboubacarhema94/assets/Kenya/GFSAD30AFCE_2015_KEN_4")
PAG_KEN_mosaic = ee.Image("projects/ee-aboubacarhema94/assets/Kenya/PAG_KEN_mosaic")
ImperviousSurfaceKEN = ee.Image("projects/ee-aboubacarhema94/assets/Kenya/KEN_gmis_impervious_surface_percentage_geographic_30m")
GAUL_adm1 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Kenya/gadm41_KEN_1")
GAUL_adm0 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Kenya/gadm41_KEN_0")
geometry = ee.Geometry.Polygon(
        [[[33.487619647710574, 5.262415464986558],
          [33.487619647710574, -5.182074987365432],
          [43.309396991460574, -5.182074987365432],
          [43.309396991460574, 5.262415464986558]]]),
ProtectedAreaPoints = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Kenya/ProtectedAreaPoints"),
ProtectedAreaPolygons = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Kenya/ProtectedAreaPolygons")

In [26]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////

#//IFPRI- Calculate Productive Agriculture  
#//December 2022
#// 
#//Description: Scripts Aggregates Cleared Forest and Cropland datasets, masking  permanent water,
          #//   and imprevious surfaces to identify arable land in Kenya at 30m resolution
          
          


#//////////////////////////////////////////////////////////////////////////////////////////////////////
#//Variables 

adm0_name = "Kenya"
deforestCompYear = 15; #//Last two digits of year only
waterOccuring = 10 #//Set water occuring less than this value (in %) of the time, it is the frequency with which water was present.

#//Get AOI of Interest
#////Import GADM data for Kenya (https://gadm.org/data.html)
adm0_AOI = GAUL_adm0
adm2_AOI = GAUL_adm1

# Import the Folium library.
import folium
# Define a method for displaying Earth Engine image tiles to folium map.

# Define the center of our map.
lat, lon = 1, 35

my_map = folium.Map(location=[lat, lon], zoom_start=6)


#//////////////////////////////////////////////////////////////////////////////////////////////////////

In [16]:
#//////////////////////////////////////////////////////////////////////////////////////////////////////

#//Imported GFSAD30AFCE Rasters --> 30m 
#//Downloaded from EarthExplorer over Kenya
#  //Download URL: https://search.earthdata.nasa.gov/search
#  //Dataset URL: https://lpdaac.usgs.gov/products/gfsad30afcev001/
GFSAD_1 = GFSAD30AFCE_2015_KEN_1
GFSAD_2 = GFSAD30AFCE_2015_KEN_2
GFSAD_3 = GFSAD30AFCE_2015_KEN_3
GFSAD_4 = GFSAD30AFCE_2015_KEN_4
#//CLip Images to Kenya Extent --> Mask 
clipGFSAD_1 = GFSAD_1.clip(adm0_AOI)

clipGFSAD_2 = GFSAD_2.clip(adm0_AOI)
clipGFSAD_3 = GFSAD_3.clip(adm0_AOI)
clipGFSAD_4 = GFSAD_4.clip(adm0_AOI)
#//Create Image Collection so it can be mosaiced together --> use quality mosaic to get highest 
GFSAD = ee.ImageCollection([clipGFSAD_1, clipGFSAD_2, clipGFSAD_3, clipGFSAD_4]).mosaic()





# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 2,
  'palette': ['red','yellow' , 'green']}    
    
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
# Add the elevation model to the map object.
my_map.add_ee_layer(GFSAD, vis_params,  adm0_name +  'GFSAD30AFCE Rasters')
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [28]:

#//Mask to select Cropland (band b1 = 2)
aoiCropland = GFSAD.eq(2).selfMask().rename("Cropland")
#//Unmask Bands Setting values to zero
arableFromCropland = aoiCropland.select('Cropland').unmask(0).clip(adm2_AOI)
vis_params_Cropland = {
  'min': 0,
  'max': 1,
  'palette': ['red', 'green']}

ee_tiles = [GFSAD, aoiCropland]

# Arrange visualization parameters inside a list.
ee_vis_params = [vis_params, vis_params_Cropland]

# Arrange layer names inside a list.
ee_tiles_names = [adm0_name +  ' GFSAD30AFCE Rasters', adm0_name +  ' Cropland in GFSAD']

# Create a new map.
my_map = folium.Map(location=[lat, lon], zoom_start=6)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

my_map
#/////////////////////////////////////////////////////////////////////////////////////////////////////